In [1]:
import pandas as pd
import seaborn as sns

# Transmission

In [2]:
def read_data(file: str) -> pd.DataFrame:
    return (pd.read_csv(file)
                .T.reset_index(drop=True).T
                .loc[:, [0, 1]]
                .rename({0: 'm/z', 1: 'intensity'}, axis=1)
           )

In [3]:
transmission = [
    read_data('qms/Transmission_ruf_e-8_280.csv')
        .assign(sensitivity=8, sem=2.8, resolution='low'),
    read_data('qms/Transmission_middle_ruf_e-8_280.csv')
        .assign(sensitivity=8, sem=2.8, resolution='middle'),
    read_data('qms/Transmission_no_ruf_e-8_280.csv')
        .assign(sensitivity=8, sem=2.8, resolution='high')
]
transmission = (
    pd.concat(transmission)
    .reset_index(drop=True)
)
transmission.resolution.astype('category', copy=False)
transmission.to_parquet('qms/transmission.parquet')
transmission.head()

,m/z,intensity,sensitivity,sem,resolution
0,0.065741,0.573588,8,2.8,low
1,0.065741,0.572612,8,2.8,low
2,0.072244,0.573588,8,2.8,low
3,0.078746,0.573262,8,2.8,low
4,0.072244,0.573262,8,2.8,low


## qualitatively

In [4]:
all_measures = [
    # first measures from previous day
    read_data('qms/Restgas_e-9_280.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='restgas', probe='previous'),
    read_data('qms/Restgas_e-10_280.csv')
        .assign(sensitivity=10, sem=2.8,
                measure='restgas', probe='previous'),
    
    # air measures
    read_data('qms/Luft_e-8_280.csv')
        .assign(sensitivity=8, sem=2.8,
                measure='all', probe='luft'),
    read_data('qms/Luft_e-9_280.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='all', probe='luft'), 
    read_data('qms/Restgas_e-8_280_nachLufteinlass.csv')
        .assign(sensitivity=8, sem=2.8,
                measure='restgas', probe='luft'),
    read_data('qms/Restgas_e-8_280_nachLufteinlass.csv')
        .assign(sensitivity=8, sem=2.8,
                measure='restgas', probe='luft'),
    
    # transission
    read_data('qms/Transmission_Messung_e-8_280_Restgas.csv')
       .assign(sensitivity=8, sem=2.8, 
               measure='restgas', probe='transmission'),
    read_data('qms/Transmission_Messung_e-9_280_Restgas.csv')
       .assign(sensitivity=9, sem=2.8, 
               measure='restgas', probe='transmission'),
    read_data('qms/Transmission_Messung_e-8_280.csv')
       .assign(sensitivity=8, sem=2.8, 
               measure='all', probe='transmission'),
    read_data('qms/Transmission_Messung_e-9_280.csv')
       .assign(sensitivity=9, sem=2.8, 
               measure='all', probe='transmission'),
    
    # quali
    read_data('qms/probemessung/Quali_Probengas_e-8_280.csv')
        .assign(sensitivity=8, sem=2.8,
                measure='all', probe='mixture'),
    read_data('qms/probemessung/Quali_Probengas_e-8_280_Restgas.csv')
        .assign(sensitivity=8, sem=2.8,
                measure='restgas', probe='mixture'),
    read_data('qms/probemessung/Quali_Probengas_e-9_280.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='all', probe='mixture'),
    read_data('qms/probemessung/Quali_Probengas_e-9_280_Restgas.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='restgas', probe='mixture'),
    
    # co2
    read_data('qms/probemessung/Eichgas_CO2_e-9_280_Reinheitskontrolle.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='all', probe='co2'),
    read_data('qms/probemessung/Eichgas_CO2_e-9_280_Reinheitskontrolle_Restgas.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='restgas', probe='co2'),
    
    # ln2
    read_data('qms/probemessung/Restgas_e-9_289_LN2.csv')
        .assign(sensitivity=9, sem=2.8,
                measure='restgas', probe='ln2'),
    
]

all_measures = (
    pd.concat(all_measures)
    .reset_index(drop=True)
)
all_measures.measure.astype('category', copy=False)
all_measures.probe.astype('category', copy=False)
all_measures.to_parquet('qms/all_measures.parquet')
all_measures.head()

,m/z,intensity,sensitivity,sem,measure,probe
0,0.072244,1.24143,9,2.8,restgas,previous
1,0.091752,1.24338,9,2.8,restgas,previous
2,0.065741,1.23070,9,2.8,restgas,previous
3,0.078746,1.24566,9,2.8,restgas,previous
4,0.085249,1.23103,9,2.8,restgas,previous


# Kalibrierung

In [5]:
cal = (pd.read_csv('qms/probemessung/Kalibrierung_CO2.csv', header=None)
      .loc[:, [1, 2, 3, 4, 5]]
      .rename({1: 'time', 2: 'pressure', 3: 'intensity', 
               4: 'time_2', 5: 'intensity_2'}, axis=1)

)
cal.loc[784:874, 'pressure'] *= 10
cal.time = pd.to_timedelta(cal.time, unit='minute')
cal.time_2 = pd.to_timedelta(cal.time_2, unit='minute')
cal.to_parquet('qms/calibration.parquet')
cal.head()

,time,pressure,intensity,time_2,intensity_2
0,0 days 00:00:02.154000,2.063539,0.643489,0 days 00:00:02.653980,0.644377
1,0 days 00:00:03.154020,2.063539,0.638651,0 days 00:00:03.654000,0.637803
2,0 days 00:00:04.153980,2.063539,0.647639,0 days 00:00:04.654020,0.641524
3,0 days 00:00:05.154000,2.063539,0.641842,0 days 00:00:05.653020,0.637501
4,0 days 00:00:06.153000,2.052205,0.647608,0 days 00:00:06.654000,0.639954
